Faz as importações neccessárias

In [3]:
import os
import re
import json
import spacy
from PyPDF2 import PdfReader
from googletrans import Translator


nlp = spacy.load("pt_core_news_sm")
translator = Translator(service_urls=['translate.googleapis.com'])

EMAIL_RE = re.compile(r'\b[\w\.-]+@[\w\.-]+\.\w{2,4}\b', re.IGNORECASE)

Funções para extrair dados dos textos

In [7]:
def clean_text(text):
    '''
    Remove: references style[1], \n
    :param text:
    :return:
    '''
    pattern = r'\d*https?://\S+|[\w.+-]+@[\w-]+\.[\w.-]+'
    text = text.replace("\u200b", "")
    text = text.replace('- ', '')
    text = text.replace(' -', '')
    text = text.replace('\n', '')
    text = re.sub(r'([A-Za-z])-\s+([A-Za-z])', r'\1\2', text)
    text = re.sub(r'\[[^\]]+\]', '', text)
    text = re.sub(r'\d*https?://\S+', '', text)
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(pattern, '', text)
    text = re.sub(r'(?:REFER[EÊ]NCIAS?|REFERENCES).*$', '', text, flags=re.DOTALL).strip()
    # text = re.sub(r'\d+', '', text)
    return text

def extract_title(text: str) -> str:
    lines = text.strip().split("\n")
    lines = [line.strip() for line in lines if line.strip()]
    return lines[0] if lines else ""

def extract_authors(text: str) -> list:
    lines = text.split('\n')
    # mantém linhas não-vazias
    lines = [l.strip() for l in lines if l.strip()]

    # 1) usa spaCy NER para extrair todos os PERSON
    doc = nlp(text)
    persons = [ent.text for ent in doc.ents if ent.label_ == "PER"]
    # remove duplicatas, mantendo ordem
    seen = set()
    persons = [p for p in persons if not (p in seen or seen.add(p))]

    authors = []
    for name in persons:
        # acha índice da linha onde aparece esse nome
        idx = next((i for i,line in enumerate(lines) if name in line), None)
        afiliacao = ""
        email = ""
        if idx is not None:
            # olha nas próximas 3 linhas por afiliação e e-mail
            for j in range(idx+1, min(idx+4, len(lines))):
                ln = lines[j]
                # se contiver “universidade” ou “departamento”
                if "universidade" in ln.lower() or "departamento" in ln.lower():
                    afiliacao = ln
                # se encontrar e-mail
                m = EMAIL_RE.search(ln)
                if m:
                    email = m.group(0)
        authors.append({
            "nome": name,
            "afiliacao": afiliacao,
            "orcid": email
        })
    return authors

def extract_abstract(text: str) -> str:
    # Agora busca por "RESUMO" (depois da tradução)
    m = re.search(r'(?i)resumo\s*[:\-]?\s*(.+?)\s*(palavras[- ]?chave|1 introdução|introdução)', text, re.DOTALL)
    return m.group(1).strip() if m else ""

def extract_keywords(text: str) -> list:
    m = re.search(r'(?i)palavras[- ]?chave\s*[:\-]?\s*(.+?)\s*(1 introdução|introdução)', text, re.DOTALL)
    if not m:
        return []
    keywords_raw = m.group(1).strip()
    return [kw.strip() for kw in re.split(r'[;,]', keywords_raw) if kw.strip()]

def extract_referencias(text: str) -> list:
    m = re.search(r'(?i)referências\s*(.+)', text, re.DOTALL)
    if not m:
        return []
    refs = [r.strip() for r in re.split(r'\n+', m.group(1)) if r.strip()]
    return refs


In [9]:
import requests


def translate_long_text(text, dest='en', chunk_size=4500):
    """
    Translates text using Google Translate API directly
    """
    translated = []

    for start in range(0, len(text), chunk_size):
        piece = text[start:start + chunk_size]

        url = "https://translate.googleapis.com/translate_a/single"
        params = {
            "client": "gtx",
            "sl": "auto",
            "tl": dest,
            "dt": "t",
            "q": piece
        }

        response = requests.get(url, params=params)
        if response.status_code == 200:
            translations = response.json()[0]
            translated_piece = ''.join([t[0] for t in translations if t[0]])
            translated.append(translated_piece)

    return ' '.join(translated)

Função para processar os pdf e extrair o corpus

In [10]:
def process_all_pdfs(input_dir: str, output_dir: str):
    os.makedirs(output_dir, exist_ok=True)

    files = [f for f in os.listdir(input_dir) if f.lower().endswith('.pdf')]

    print(f"Encontrados {len(files)} arquivos para processar.")

    corpus = []

    for fname in files:
        try:
            path = os.path.join(input_dir, fname)
            print(f"\n Processando: {path}")

            name = os.path.splitext(fname)[0]

            reader = PdfReader(path)
            full_text = []
            for page in reader.pages:
                full_text.append(page.extract_text())

            full_text = '\n'.join(full_text)  
            full_text = translate_long_text(full_text, dest='pt')
        
            cleaned_text = clean_text(full_text)

            doc = nlp(cleaned_text)

            tokens = [token.text for token in doc if not token.is_punct and not token.is_space]
            pos_tags = [token.pos_ for token in doc if not token.is_punct and not token.is_space]
            lemmas = [token.lemma_ for token in doc if not token.is_punct and not token.is_space]

            artigo_json = {
                "titulo": extract_title(full_text),
                "informacoes_url": "",
                "idioma": "Português",
                "storage_key": f"files/{name}.pdf",
                "autores": extract_authors(full_text),
                "data_publicacao": "2024",  # Ajuste manual se precisar
                "resumo": extract_abstract(full_text),
                "keywords": extract_keywords(full_text),
                "referencias": extract_referencias(full_text),
                "artigo_completo": cleaned_text,
                "artigo_tokenizado": tokens,
                "pos_tagger": pos_tags,
                "lema": lemmas
            }

            corpus.append(artigo_json)
            print(f"Processado {fname} com sucesso!")

        except Exception as e:
            print(f"Erro ao processar {fname}: {str(e)}")

     # Salvar JSON
    out_path = os.path.join(output_dir, f"corpus.json")
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(corpus, f, ensure_ascii=False, indent=4)

    print("\n Processamento concluído!")

Chamar função que vai exibir processar os pdf

In [11]:
input_dir = "./files"       # Pasta onde estão seus PDFs
output_dir = "./jsons"     # Pasta onde vai salvar os JSONs

process_all_pdfs(input_dir, output_dir)


Encontrados 40 arquivos para processar.

 Processando: ./files/985-24770-1-10-20240923.pdf
Processado 985-24770-1-10-20240923.pdf com sucesso!

 Processando: ./files/985-24763-2-10-20241004.pdf
Processado 985-24763-2-10-20241004.pdf com sucesso!

 Processando: ./files/985-24772-1-10-20240923.pdf
Processado 985-24772-1-10-20240923.pdf com sucesso!

 Processando: ./files/985-24742-1-10-20240923.pdf
Processado 985-24742-1-10-20240923.pdf com sucesso!

 Processando: ./files/985-24756-1-10-20240923.pdf
Processado 985-24756-1-10-20240923.pdf com sucesso!

 Processando: ./files/985-24758-1-10-20240923.pdf
Processado 985-24758-1-10-20240923.pdf com sucesso!

 Processando: ./files/985-24766-1-10-20240923.pdf
Processado 985-24766-1-10-20240923.pdf com sucesso!

 Processando: ./files/985-24753-2-10-20241004.pdf
Processado 985-24753-2-10-20241004.pdf com sucesso!

 Processando: ./files/985-24769-1-10-20240923.pdf
Processado 985-24769-1-10-20240923.pdf com sucesso!

 Processando: ./files/985-24760-